# Prompt Formatting

## Introduction

Prompt engineering is fundamental to Gen AI. Gen AI models are very sensitive to the precise language, structure, style, and formatting of prompts. 

For general guidance on writing strong prompts, we recommend consulting documentation on best practices from leading model providers:

- OpenAI
    - [Prompt Engineering](https://platform.openai.com/docs/guides/prompt-engineering)
    - [Best Practices for Prompt Engineering with the OpenAI API](https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-the-openai-api)
    - [Prompt Examples](https://platform.openai.com/docs/examples)
- Anthropic
    - [Prompt Engineering](https://docs.anthropic.com/en/docs/prompt-engineering)
    - [Prompt Library](https://docs.anthropic.com/en/prompt-library/library)

In this guide, we introduce a few 'gotchas' and tips for ensuring your prompts are formatted correctly in notebooks and Python scripts.

## Excess White Space

It's a common error to define system prompts as a multi-line string within a python function, forgetting about the white space that is introduced before new lines. LLMs are sensitive to white space, and so this pattern could introduce unintended noise to your prompts:

In [1]:
def create_prompt(exclamation):

    prompt = f"""Line one.
    Line two.
    {exclamation}!
    """
    
    return prompt

prompt = create_prompt("Uh-Oh")

print(prompt)

Line one.
    Line two.
    Uh-Oh!
    


Let's look at the raw string to see what is going on:

In [2]:
prompt

'Line one.\n    Line two.\n    Uh-Oh!\n    '

We can see that definition of the prompt within the `create_prompt` function has resulted in some unexpected white space at the start and end of lines. Specifically:

1. The conventional 4-space Python function indentation has resulted in extra white space at the beginning of lines one and two.
2. Extra white space is also present at the end of the third line.

What can we do?

## Best Practices

Before introducing some specific options that avoid unwanted white space, it is worth highlighting the single most important best practice:

> ALWAYS PRINT YOUR PROMPTS!!!

If you don't print your prompts, it is very easy to miss unwanted white space in your prompts.

With that established, let's look at a few ways you can define the prompt above in a way that avoids undesirable white space.

### Option 1: Constants

Part of the problem is that multi-line strings defined within Python functions must be indented to conform with Python coding conventions, but this results in extra white space which will be seen by the LLM. 

One solution is to define the prompt as a constant outside of the Python function. Note that we must also use backslashes to escape the extra white space which would otherwise be prepended to the first line and appended to the second line due to the implicit newlines:

In [3]:
PROMPT = """\
Line one.
Line two.
{exclamation}!\
"""

def create_prompt(exclamation):

    prompt = PROMPT.format(exclamation=exclamation)
    
    return prompt

prompt = create_prompt("Nice")

print(prompt)

Line one.
Line two.
Nice!


In [4]:
prompt

'Line one.\nLine two.\nNice!'

### Option 2: String concat

Another option is to use string concatenation within the Python function. This might make sense for very short prompts. Note that you must explicitly define newlines with this approach:

In [5]:
def create_prompt(exclamation):

    prompt = (
        "Line one.\n"
        "Line two.\n"
        f"{exclamation}!"
    )
    
    return prompt

prompt = create_prompt("Also works")

print(prompt)

Line one.
Line two.
Also works!


In [6]:
prompt

'Line one.\nLine two.\nAlso works!'

### Option 3: Load from YAML

If you have a lot of long prompts in your testing suite, it may be best to store them in a config instead of defining them within Python scripts or notebooks.

[YAML](https://pyyaml.org/wiki/PyYAMLDocumentation) is a config format that is popular, powerful, and easy to use. Below is a quick example of how you can load prompts from a YAML file. 

We have a YAML file located in `./data/prompt_config.yml` which contains the following text:

```
yaml_example: |
  Line one.
  Line two.
  {exclamation}!
```

The vertical bar (`|`) indicates that newlines in the YAML file should be taken literally. 

We can load our YAML file like this:

In [7]:
import yaml

def load_prompt_config():
    cfg_path = './data/prompt_config.yml'
    with open(cfg_path, 'r') as file:
        prompts = yaml.safe_load(file)
    return prompts

prompt_config = load_prompt_config()

prompt_config

{'yaml_example': 'Line one.\nLine two.\n{exclamation}!'}

And confirm that we get the desired result:

In [8]:
def create_prompt(exclamation):

    prompt = prompt_config['yaml_example'].format(exclamation=exclamation)
    
    return prompt

prompt = create_prompt('Yay')

print(prompt)

Line one.
Line two.
Yay!


In [9]:
prompt

'Line one.\nLine two.\nYay!'

## Concluding Remarks

We note that there are many other options for defining prompts outside your Python scripts, and the best solution depends on your project needs and personal preference. Other file formats which can be used to store prompts include TXT, CSV, JSON, TOML, and many others. 

There is no single right or wrong way to store prompts. The most important thing is to be consistent, and when you are developing... ALWAYS PRINT YOUR PROMPTS!